# Edit Your Expectation Suite
Use this notebook to recreate and modify your expectation suite:

**Expectation Suite Name**: `milling_machine_data_quality`

In [2]:
import datetime

import pandas as pd

import great_expectations as gx
import great_expectations.jupyter_ux
from great_expectations.core.batch import BatchRequest
from great_expectations.checkpoint import SimpleCheckpoint
from great_expectations.exceptions import DataContextError

context = gx.get_context()

# Note that if you modify this batch request, you may save the new version as a .json file
#  to pass in later via the --batch-request option
batch_request = {'datasource_name': 'milling_machine_data', 'data_connector_name': 'default_inferred_data_connector_name', 'data_asset_name': 'data_split_6.csv', 'limit': 1000}


# Feel free to change the name of your suite here. Renaming this will not remove the other one.
expectation_suite_name = "milling_machine_data_quality"
try:
    suite = context.get_expectation_suite(expectation_suite_name=expectation_suite_name)
    print(f'Loaded ExpectationSuite "{suite.expectation_suite_name}" containing {len(suite.expectations)} expectations.')
except DataContextError:
    suite = context.add_expectation_suite(expectation_suite_name=expectation_suite_name)
    print(f'Created ExpectationSuite "{suite.expectation_suite_name}".')


validator = context.get_validator(
    batch_request=BatchRequest(**batch_request),
    expectation_suite_name=expectation_suite_name
)
column_names = [f'"{column_name}"' for column_name in validator.columns()]
print(f"Columns: {', '.join(column_names)}.")
validator.head(n_rows=5, fetch_all=False)

2025-01-15T18:54:05+0100 - INFO - Great Expectations logging enabled at 20 level by JupyterUX module.
2025-01-15T18:54:05+0100 - INFO - FileDataContext loading fluent config
2025-01-15T18:54:05+0100 - INFO - Loading 'datasources' ->
[]
Loaded ExpectationSuite "milling_machine_data_quality" containing 0 expectations.


Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

Columns: "Product ID", "Type", "Air temperature [K]", "Process temperature [K]", "Rotational speed [rpm]", "Torque [Nm]", "Tool wear [min]".


Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min]
0,L49700,L,NaN,308.6,1535.0,41.2,NaN
1,L49701,L,298.8,308.6,1584.0,34.4,42
2,M17382,M,298.8,308.7,1543.0,38.2,error
3,L49703,L,298.8,308.7,1635.0,35.2,47
4,M17384,M,298.8,308.7,1446.0,45.5,49


## Create & Edit Expectations


Add expectations by calling specific expectation methods on the `validator` object. They all begin with `.expect_` which makes autocompleting easy using tab.

Because you selected interactive mode, you are now creating or editing an Expectation Suite with validator feedback from the sample batch of data that you specified (see `batch_request`).

Note that if you select manual mode you may still create or edit an Expectation Suite directly, without feedback from the `validator`. See our documentation for more info and examples: [How to create a new Expectation Suite without a sample batch](https://docs.greatexpectations.io/docs/guides/expectations/how_to_create_and_edit_expectations_based_on_domain_knowledge_without_inspecting_data_directly).



You can see all the available expectations in the **[expectation gallery](https://greatexpectations.io/expectations)**.

In [5]:
validator.expect_table_columns_to_match_ordered_list(
    ["Product ID", "Type", "Air temperature [K]", "Process temperature [K]", 
     "Rotational speed [rpm]", "Torque [Nm]", "Tool wear [min]"]
)
validator.expect_column_values_to_not_be_null("Product ID")
validator.expect_column_values_to_be_unique("Product ID")
validator.expect_column_values_to_match_regex("Product ID", "^[LMH]\\d+$")
validator.expect_column_values_to_not_be_null("Type")
# 2. Expect Type to be one of the predefined categories
validator.expect_column_values_to_be_in_set(
    column="Type",
    value_set=["L", "M", "H"]
)
# 3. Expect Air temperature [K] to be within a realistic range
validator.expect_column_values_to_be_between(
    column="Air temperature [K]",
    min_value=293,
    max_value=313,
    mostly=0.95  # Allow for up to 5% exceptions
)
# 4. Expect Process temperature [K] to be within a realistic range (numeric only)
validator.expect_column_values_to_be_of_type(column="Process temperature [K]", type_="float")
validator.expect_column_values_to_not_be_null(column="Air temperature [K]", mostly=0.9)
validator.expect_column_values_to_be_between(column="Air temperature [K]", min_value=298.0, max_value=310.0, mostly=0.9)
validator.expect_column_values_to_not_be_null(column="Rotational speed [rpm]", mostly=0.9)
# 5. Expect Rotational speed [rpm] to be within a realistic range
validator.expect_column_values_to_be_between(
    column="Rotational speed [rpm]",
    min_value=500,
    max_value=10000,
    mostly=0.98  # Allow for up to 2% exceptions
)
validator.expect_column_values_to_not_be_null(column="Torque [Nm]", mostly=0.9)
# 6. Expect Torque [Nm] to be within a realistic range
validator.expect_column_values_to_be_between(
    column="Torque [Nm]",
    min_value=10,
    max_value=80,
    mostly=0.95
)
validator.expect_column_values_to_not_be_null(column="Tool wear [min]", mostly=0.9)

# 8. Expect no null values in required columns
required_columns = [
    "Product ID", 
    "Type", 
    "Air temperature [K]", 
    "Process temperature [K]", 
    "Rotational speed [rpm]"
]

for col in required_columns:
    validator.expect_column_values_to_not_be_null(column=col)

# 10. Custom handling for errors (e.g., "error" values in Tool wear [min])
validator.expect_column_values_to_not_match_regex(
    column="Tool wear [min]",
    regex=r"error"
)


Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": false,
  "result": {
    "element_count": 420,
    "unexpected_count": 40,
    "unexpected_percent": 10.471204188481675,
    "partial_unexpected_list": [
      "error",
      "error",
      "error",
      "error",
      "error",
      "error",
      "error",
      "error",
      "error",
      "error",
      "error",
      "error",
      "error",
      "error",
      "error",
      "error",
      "error",
      "error",
      "error",
      "error"
    ],
    "missing_count": 38,
    "missing_percent": 9.047619047619047,
    "unexpected_percent_total": 9.523809523809524,
    "unexpected_percent_nonmissing": 10.471204188481675
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

### Table Expectation(s)

No table level expectations are in this suite. Feel free to add some here.

They all begin with `validator.expect_table_...`.


### Column Expectation(s)

No column level expectations are in this suite. Feel free to add some here.

They all begin with`validator.expect_column_...`.


## Review & Save Your Expectations

Let's save the expectation suite as a JSON file in the `great_expectations/expectations` directory of your project.

Let's now rebuild your Data Docs, which helps you communicate about your data with both machines and humans.

In [6]:
print(validator.get_expectation_suite(discard_failed_expectations=False))
validator.save_expectation_suite(discard_failed_expectations=False)

checkpoint_config = {
    "class_name": "SimpleCheckpoint",
    "validations": [
        {
            "batch_request": batch_request,
            "expectation_suite_name": expectation_suite_name
        }
    ]
}
checkpoint = SimpleCheckpoint(
    f"{validator.active_batch_definition.data_asset_name}_{expectation_suite_name}",
    context,
    **checkpoint_config
)
checkpoint_result = checkpoint.run()

context.build_data_docs()

validation_result_identifier = checkpoint_result.list_validation_result_identifiers()[0]
context.open_data_docs(resource_identifier=validation_result_identifier)

2025-01-15T19:00:33+0100 - INFO - 	16 expectation(s) included in expectation_suite.
{
  "expectation_suite_name": "milling_machine_data_quality",
  "ge_cloud_id": null,
  "expectations": [
    {
      "expectation_type": "expect_table_columns_to_match_ordered_list",
      "kwargs": {
        "column_list": [
          "Product ID",
          "Type",
          "Air temperature [K]",
          "Process temperature [K]",
          "Rotational speed [rpm]",
          "Torque [Nm]",
          "Tool wear [min]"
        ]
      },
      "meta": {}
    },
    {
      "expectation_type": "expect_column_values_to_not_be_null",
      "kwargs": {
        "column": "Product ID"
      },
      "meta": {}
    },
    {
      "expectation_type": "expect_column_values_to_be_unique",
      "kwargs": {
        "column": "Product ID"
      },
      "meta": {}
    },
    {
      "expectation_type": "expect_column_values_to_match_regex",
      "kwargs": {
        "column": "Product ID",
        "regex": "^[L

Calculating Metrics:   0%|          | 0/78 [00:00<?, ?it/s]